In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import patches
import cartopy.crs as ccrs
import cartopy.feature
import paraguayfloodspy.visualize as viz
%matplotlib inline

## Setup

Load the parameters of the data

In [ ]:
wt = pd.read_csv('../data/derived/WeatherTypes.csv', index_col='time', parse_dates=True)
wt['T'] = wt.index.to_period("M")
wt = wt.groupby(['T', 'wtype']).size().unstack()
wt.sort_values(4, ascending=False).head()

In [ ]:
def lon_360to180(longitude):
    x = longitude.copy()
    x[np.where(x > 180)] = x[np.where(x > 180)] - 360
    return(x)

In [ ]:
sst = xr.open_dataarray('../data/accessed/sst.nc', decode_times=False).sel(Y = slice(-80, 15))
sst['T'] = pd.PeriodIndex(freq='m', start='1981-11', periods=sst['T'].size).to_timestamp()

# Months with Most WT 4

In [ ]:
n_months = 15
times = wt.sort_values(4, ascending=False).head(n_months).index.to_timestamp()

In [ ]:
composite_wt4 = sst.sel(T = np.in1d(sst['T'], times)).mean(dim='T')

In [ ]:
def dipole_box():
    rec = patches.Rectangle(
        (325, -40), 25, 30,
        color = 'black', label = 'Box',
        fill = None, linewidth=2,
        transform = ccrs.PlateCarree()
    )
    return(rec)

In [ ]:
plt.figure(figsize=(14,5))
ax = plt.axes(projection=ccrs.PlateCarree(-100))
composite_wt4.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=np.linspace(-0.6, 0.6, 13), 
                            cmap='PuOr_r', cbar_kwargs={'label': 'SST Anomaly [Degrees C]'})
ax.coastlines()
ax.set_title('Composite: the {} NDJF Months With Most WT4'.format(n_months))
ax.set_xlim([-120, 120])
ax.add_patch(dipole_box())
plt.savefig('../figs/WT4_Monthly_Composite.pdf'.format(n_months), bbox_inches='tight')